# Run Code and View the output:
    Run time is too slow and output is incorrect.
    Code is modified to run in notebook and limit total number of files written
****
## scipy image: cleaning, segmentation, etc.
[API Reference: scikit-image](https://scikit-image.org/docs/dev/api/api.html) <br>
****
[Home: scikit-image aka skimage](https://scikit-image.org/) <br>
[Documentation: scikit-image](https://scikit-image.org/docs/stable/) <br>
[scikit-image on GitHub](https://github.com/scikit-image/scikit-image) <br>
[User Guide: scikit-image](https://scikit-image.org/docs/dev/user_guide) <br>
[Founder-tutor](https://bids.berkeley.edu/resources/videos/scikit-image-image-analysis-python-intermediate) <br>
****
## OpenSlide:
[OpenSlide api](https://openslide.org/api/python/) <br>
[openslide github](https://github.com/openslide/openslide-python) <br>
[openslide dot org](https://openslide.org/) <br>

In [1]:
import os
import time
output_dir = '../../DigiPath_MLTK_data/v0_out_to_test'
if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)
    
OUTPUT_PATH = output_dir
data_dir = '../../DigiPath_MLTK_data/Aperio'
files_list = ['CMU-1-Small-Region.svs', 'CMU-1.svs']
SVS = os.path.join(data_dir, files_list[0])
print('Working with file:', SVS, '\nOutput path:', OUTPUT_PATH)

Working with file: ../../DigiPath_MLTK_data/Aperio/CMU-1-Small-Region.svs 
Output path: ../../DigiPath_MLTK_data/v0_out_to_test


In [2]:
# %load '../../../Mayomics-Research/cc/wsi-sampler/sampler.py'
#                                      <0><0>
MAXPATCHESTOWRITE = 1000
#                                      <0><0>

import time
import glob
import math
import os

import numpy as np
import openslide
from skimage.filters import threshold_otsu

start_time = time.time()
#
# -----------------------------------------------------------
# Create function
def get_thumbnail(img, x_level0, y_level0, patch_size, output_path):
    """

    :param img:
    :param x_level0:
    :param y_level0:
    :param patch_size:
    :param output_path:
    :return:
    """
    patch = img.read_region((x_level0, y_level0), 0, (patch_size, patch_size))
    patch = patch.convert('RGB')
    fname = img.properties['aperio.Filename'].replace(' ', '_')
    fname += '_' + str(x_level0)
    fname += '_' + str(y_level0)
    fname += '_' + '0'
    fname += '_' + str(patch_size)
    fname += '.jpg'
    patch_arr = np.array(patch.convert('L'))
    # print('Getting close: {}'.format(np.average(patch_arr)))
    if np.average(patch_arr) < 230:
        global num_patches
        num_patches += 1
        patch.save(os.path.join(output_path, fname))


def process_svs(SVS,
                normalization_factor=1000,
                patch_size=512,
                buffer=10,
                output_path=OUTPUT_PATH):
    img = openslide.OpenSlide(SVS)

    global num_patches
    num_patches = 0

    # Normalize the thumbnail to the actual slide size
    x_max, y_max = img.dimensions
    thumbnail = img.get_thumbnail((normalization_factor, normalization_factor))

    grey_thumbnail = np.array(thumbnail.convert("L"))
    thresh = threshold_otsu(grey_thumbnail)
    mask = np.array(grey_thumbnail) < thresh

    # how many pixels in the raw image per pixel in mask
    x_num_orgPix_per_thumbPix = math.ceil(x_max / mask.shape[0])
    y_num_orgPix_per_thumbPix = math.ceil(y_max / mask.shape[1])
    # print(x_num_orgPix_per_thumbPix, y_num_orgPix_per_thumbPix)

    # Find out how many pixels in image mask to count as a patch in original
    num_x_mask_pixels_per_rawPatch = math.ceil(patch_size / x_num_orgPix_per_thumbPix)
    num_y_mask_pixels_per_rawPatch = math.ceil(patch_size / y_num_orgPix_per_thumbPix)
    # print(num_x_mask_pixels_per_rawPatch, num_y_mask_pixels_per_rawPatch)

    mask_x, mask_y = mask.shape
    x_mask_prev = 0

    # Iterate through the mask to identify positive pixels
    for x in range(buffer, mask_x - buffer):
        x_mask_window = x + num_x_mask_pixels_per_rawPatch
        if x_mask_window <= x_mask_prev:
            continue
        y_mask_prev = 0
        for y in range(buffer, mask_y - buffer):
            y_mask_window = y + num_y_mask_pixels_per_rawPatch
            # print('Evaluate: {} {} & {}'.format(y, y_mask_window, y_mask_prev))
            if y_mask_window <= y_mask_prev:
                continue
            if y % 100 == 0:
                print('X: {}\tY:{} of {} with total of {} so far'.format(x, y, 
                                                                mask.shape, 
                                                                num_patches), end='\r',
                                                                flush=True)
                
            if np.sum(mask[x:x_mask_window, y:y_mask_window]) > 0:
                # convert mask coordinates to level0 coordinates
                x_level0 = x * x_num_orgPix_per_thumbPix
                y_level0 = y * y_num_orgPix_per_thumbPix
                get_thumbnail(img, x_level0, y_level0, patch_size, output_path)
                
                #                                      <0><0>   Limit number of files
                if num_patches >= MAXPATCHESTOWRITE:
                    print('\n\n\t{} num_patched out\n\n'.format(num_patches))
                    return
                #                                      <0><0>

            # print('yamsk windoe: {}'.format(y_mask_window))
            y_mask_prev = y_mask_window
        x_mask_prev = x_mask_window

    print('Printed {} from {}'.format(num_patches, SVS))

num_patches = 0
"""
# if __name__ == '__main__':
#     for SVS in glob.glob('/data/biliary/svs/negative/*svs'):
#         process_svs(SVS)
"""
print('processing: {}'.format(SVS))
process_svs(SVS)
print('total running time: %0.3f'%(time.time() - start_time))

processing: ../../DigiPath_MLTK_data/Aperio/CMU-1-Small-Region.svs
X: 37	Y:600 of (1000, 748) with total of 992 so far

	1000 num_patched out


total running time: 99.108


In [3]:
f_ext_list = ['.jpg', '.png']
file_name_list = []
for f in os.listdir(output_dir):
    ff = os.path.join(output_dir, f)
    if os.path.isfile(ff):
        _, f_ext = os.path.splitext(ff)
        if f_ext in f_ext_list:
            file_name_list.append(ff)
print(len(file_name_list), file_name_list[0])        

1000 ../../DigiPath_MLTK_data/v0_out_to_test/CMU-1_96_2828_0_512.jpg
